In [3]:
import random
import pandas as pd


# Função para criar uma transação
def create_transaction(client_id, transacao_id, fraude=False):
    renda_anual = random.randint(20000, 150000)  # Renda anual em dólares
    historico_credito = round(
        random.uniform(0.3, 1.0), 2
    )  # Histórico de crédito entre 0.3 (ruim) e 1.0 (excelente)
    valor = random.randint(50, 50000)  # Valor da transação em dólares
    tipo_transacao = random.choice(["presencial", "online"])
    local_transacao = random.choice(
        ["Araraquara", "São Carlos", "Ribeirão Preto", "Matão", "São Paulo", "Bahia"]
    )
    categoria_comercio = random.choice(
        ["supermercado", "eletronicos", "restaurante", "luxo", "vestuário"]
    )
    canal_autenticacao = random.choice(["chip", "online"])
    frequencia_transacoes_24h = random.randint(1, 10)
    distancia_localizacao = random.randint(0, 10000)  # Distância em quilômetros
    tentativas_falhas_ultimas_24h = random.randint(0, 5)
    hora_transacao = f"{random.randint(0, 23):02d}:{random.randint(0, 59):02d}"
    data_hora = f"2024-08-27T{hora_transacao}:00"

    # Regras para marcar como fraude
    if fraude:
        if random.random() < 0.5:
            valor = random.randint(
                20000, 50000
            )  # Valor muito alto para forçar a fraude
        if random.random() < 0.5:
            distancia_localizacao = random.randint(
                5000, 10000
            )  # Localização distante para forçar a fraude
        if random.random() < 0.5:
            historico_credito = round(
                random.uniform(0.3, 0.5), 2
            )  # Histórico de crédito ruim para forçar a fraude

    return {
        "cartao_id": f"c{transacao_id}",
        "cliente_id": client_id,
        "idade": random.randint(18, 70),
        "renda_anual": renda_anual,
        "historico_credito": historico_credito,
        "transacao_id": f"t{transacao_id}",
        "valor": valor,
        "data_hora": data_hora,
        "tipo_transacao": tipo_transacao,
        "local_transacao": local_transacao,
        "categoria_comercio": categoria_comercio,
        "canal_autenticacao": canal_autenticacao,
        "frequencia_transacoes_24h": frequencia_transacoes_24h,
        "distancia_localizacao": distancia_localizacao,
        "tentativas_falhas_ultimas_24h": tentativas_falhas_ultimas_24h,
        "hora_transacao": hora_transacao,
        "fraudulenta": 1 if fraude else 0,
    }


dataset = []

for i in range(800000):
    dataset.append(create_transaction(f"cl{i+1}", i + 1, fraude=True))

for i in range(800000, 1000000):
    dataset.append(create_transaction(f"cl{i+1}", i + 1))

df = pd.DataFrame(dataset)

df.head(10)


,cartao_id,cliente_id,idade,renda_anual,historico_credito,transacao_id,valor,data_hora,tipo_transacao,local_transacao,categoria_comercio,canal_autenticacao,frequencia_transacoes_24h,distancia_localizacao,tentativas_falhas_ultimas_24h,hora_transacao,fraudulenta
0,c1,cl1,56,78142,0.87,t1,1729,2024-08-27T06:20:00,presencial,Ribeirão Preto,restaurante,online,8,4170,5,06:20,1
1,c2,cl2,65,51253,0.35,t2,49878,2024-08-27T17:03:00,online,Araraquara,restaurante,chip,6,5530,5,17:03,1
2,c3,cl3,50,95312,0.44,t3,43014,2024-08-27T14:55:00,presencial,Araraquara,vestuário,chip,4,7392,2,14:55,1
3,c4,cl4,22,94480,0.83,t4,24328,2024-08-27T23:50:00,presencial,São Carlos,supermercado,online,6,5816,3,23:50,1
4,c5,cl5,54,126026,0.34,t5,44120,2024-08-27T08:36:00,online,Bahia,eletronicos,online,9,4625,5,08:36,1
5,c6,cl6,43,84579,0.31,t6,46412,2024-08-27T04:13:00,presencial,Bahia,supermercado,online,7,8355,1,04:13,1
6,c7,cl7,67,72890,0.31,t7,24056,2024-08-27T11:54:00,online,Ribeirão Preto,supermercado,online,6,1092,2,11:54,1
7,c8,cl8,68,53845,0.44,t8,37196,2024-08-27T05:51:00,online,Bahia,vestuário,online,10,8086,3,05:51,1
8,c9,cl9,50,117970,0.34,t9,27715,2024-08-27T03:01:00,presencial,Araraquara,supermercado,chip,2,8776,4,03:01,1
9,c10,cl10,68,118710,0.80,t10,37391,2024-08-27T08:45:00,online,Araraquara,eletronicos,chip,2,9480,5,08:45,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import joblib

In [3]:
def hour_to_minutes(hour):
    h, m = map(int, hour.split(':'))
    return h * 60 + m

df['hora_transacao'] = df['hora_transacao'].apply(hour_to_minutes)

df.head(10)

,cartao_id,cliente_id,idade,renda_anual,historico_credito,transacao_id,valor,data_hora,tipo_transacao,local_transacao,categoria_comercio,canal_autenticacao,frequencia_transacoes_24h,distancia_localizacao,tentativas_falhas_ultimas_24h,hora_transacao,fraudulenta
0,c1,cl1,20,147150,0.42,t1,24788,2024-08-27T07:32:00,presencial,São Carlos,luxo,chip,2,7832,3,452,1
1,c2,cl2,24,38368,0.42,t2,24848,2024-08-27T18:46:00,presencial,Ribeirão Preto,restaurante,online,6,1767,5,1126,1
2,c3,cl3,36,146420,0.69,t3,33105,2024-08-27T08:16:00,presencial,Matão,luxo,chip,2,6009,2,496,1
3,c4,cl4,34,74139,0.61,t4,16987,2024-08-27T14:28:00,presencial,São Carlos,restaurante,chip,9,7783,5,868,1
4,c5,cl5,28,52733,0.44,t5,41711,2024-08-27T03:34:00,presencial,Matão,restaurante,chip,10,7497,3,214,1
5,c6,cl6,29,90102,0.56,t6,13439,2024-08-27T05:42:00,online,São Paulo,restaurante,chip,2,8362,1,342,1
6,c7,cl7,48,50781,0.86,t7,15898,2024-08-27T07:52:00,presencial,Matão,eletronicos,online,8,4256,4,472,1
7,c8,cl8,30,23554,0.37,t8,4707,2024-08-27T06:01:00,online,São Carlos,supermercado,chip,2,8575,0,361,1
8,c9,cl9,47,131458,0.39,t9,28006,2024-08-27T16:58:00,presencial,Bahia,luxo,chip,6,1612,2,1018,1
9,c10,cl10,29,125816,0.49,t10,27034,2024-08-27T00:42:00,presencial,São Paulo,luxo,online,9,9542,2,42,1


## Dividindo os dados em conjuntos para treinamento e teste de desempenho

In [4]:
X = df.drop(columns=["fraudulenta"])
y = df["fraudulenta"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Normalizando os dados para melhorar a precisão do modelo.

No contexto de machine learning, as entradas usadas pelo modelo para realizar previsões são chamadas de features ou variáveis. Essas features podem ser divididas em dois tipos principais:

1. Features Numéricas:
>Definição: São variáveis que representam valores numéricos e podem ser contínuas (como idade, renda) ou discretas (como o número de tentativas de login).
> Exemplos: Idade, renda anual, e valor.
Tratamento no Código: As features numéricas são ajustadas (normalizadas) para que tenham uma média de 0 e um desvio padrão de 1 usando a ferramenta StandardScaler. Isso garante que todas as features numéricas estejam na mesma escala, o que facilita o processo de treinamento do modelo.
2. Features Categóricas:
> Definição: São variáveis que representam categorias ou rótulos, como tipos, locais, ou qualquer outra informação que seja representada por texto ou por números que indicam uma categoria.
> Exemplos: Tipo de transação, local da transação, e categoria do comércio.
Tratamento no Código: As features categóricas precisam ser convertidas em um formato que o modelo possa entender. Isso é feito através do OneHotEncoder, que transforma cada categoria em uma série de valores binários (0 ou 1), onde cada valor indica a presença ou ausência de uma determinada categoria.
Esses processos de normalização e codificação são essenciais para preparar as features de maneira adequada para o treinamento eficaz do modelo de machine learning.

In [5]:
categorical_features = [
    "tipo_transacao",
    "local_transacao",
    "categoria_comercio",
    "canal_autenticacao",
]
numerical_features = [
    "idade",
    "renda_anual",
    "historico_credito",
    "valor",
    "frequencia_transacoes_24h",
    "distancia_localizacao",
    "tentativas_falhas_ultimas_24h",
    "hora_transacao",
]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Ignorar categorias desconhecidas
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [6]:
# Treinando o modelo de regressão logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Prevendo no conjunto de teste
y_pred = model.predict(X_test)

# Avaliando o modelo
# Métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Acurácia (accuracy): {accuracy:.2f}")
print(f"Precisão (precision): {precision:.2f}")
print(f"Sensibilidade (recall): {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Acurácia (accuracy): 0.81
Precisão (precision): 0.82
Sensibilidade (recall): 0.98
F1-Score: 0.89


## Salvando o modelo treinado

Podemos salvar o modelo treinado junto com o pipeline de pré-processamento para usar na API.

In [22]:
import os

if not os.path.exists("src/ml_model"):
    os.makedirs("src/ml_model")
# Salvando o modelo e o pipeline de pré-processamento
joblib.dump(model, "src/ml_model/model_alpha_fraud_detector.pkl")
joblib.dump(preprocessor, "src/ml_model/alpha_preprocessor.pkl")

['src/ml_model/alpha_preprocessor.pkl']

## Testando o modelo treinado

Vamos usar duas transações sendo uma de cada tipo para verificar se o modelo consegue compreender as diferenças entre elas e identificar qual é qual

In [20]:
# Exemplo de transação legítima
legit_transaction = {
    "idade": 35,
    "renda_anual": 120000,
    "historico_credito": 0.9,
    "valor": 500,
    "tipo_transacao": "presencial",
    "local_transacao": "Sao Paulo",
    "categoria_comercio": "supermercado",
    "canal_autenticacao": "chip",
    "frequencia_transacoes_24h": 1,
    "distancia_localizacao": 0,  # Transação feita na localização usual
    "tentativas_falhas_ultimas_24h": 0,
    "hora_transacao": "10:00"
}

# Exemplo de transação fraudulenta
fraudulent_transaction = {
    "idade": 67,
    "renda_anual": 30000,
    "historico_credito": 0.4,
    "valor": 23000,
    "tipo_transacao": "presencial",
    "local_transacao": "Sao Paulo",
    "categoria_comercio": "eletronicos",
    "canal_autenticacao": "chip",
    "frequencia_transacoes_24h": 2,
    "distancia_localizacao": 9000,
    "tentativas_falhas_ultimas_24h": 1,
    "hora_transacao": "14:00"
}


def predict(transaction):
    # Convertendo a hora para minutos
    transaction["hora_transacao"] = hour_to_minutes(transaction["hora_transacao"])

    # Convertendo para DataFrame
    df_transaction = pd.DataFrame([transaction])

    # Aplicando o pré-processamento
    X_transaction = preprocessor.transform(df_transaction)

    # Fazendo a previsão
    result = model.predict(X_transaction)
    
    print(result[0])

    # Interpretando o resultado
    if result[0] == 1:
        print("A transação é suspeita de ser fraudulenta.")
    else:
        print("A transação parece legítima.")

predict(legit_transaction)
predict(fraudulent_transaction)

0
A transação parece legítima.
1
A transação é suspeita de ser fraudulenta.
